In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# from econtools.metrics import reg
import matplotlib 
from matplotlib.lines import Line2D

pd.set_option("display.max_columns",50)
%matplotlib inline

In [2]:
# Read in yield data
d = pd.read_csv("../data/output_data/soybeans_95_20.csv")

# Convert from bu/acre to kg/ha
d['yield_kg'] = (d['yield'] * 67.25)
d['variety_id'] = d['company_name'] + "-" + d['variety'].str.replace(" ","_").str.replace("-","_").str.upper()
d['variety_id'] = d['variety_id'].str.replace(".","")
d['variety_id'] = d['variety_id'].apply(lambda x: x.replace("__",""))

# Combine the lodging score:
d['lodging'] = d.lodging.round(1)

d['lodging_comb'] = d['lodging']
d.loc[pd.isnull(d['lodging_comb']),"lodging_comb"] = d['lodging05'].round(1)

# Mislabeled location
d['location'] = d.location.str.replace("mtmorris","mt_morris")

# Read in entries
E = pd.read_csv("../data/Illinois_Variety_Trials/entries_97_20.csv")
E['mgroup'] = E.maturity.apply(lambda x: str(x).split(".")[0])

# Read in where company participated
first_il = pd.read_csv("../data/Illinois_Variety_Trials/FIRST_or_IL.csv")
E = E.merge(first_il,how="left").drop_duplicates()

d = d.merge(E[['variety','company','year','mgroup','first_il']],
           on = ['variety','company','year'],
           how="left",
           suffixes=["","_entries"])

C:\Users\jhtchns2\AppData\Local\Temp\ipykernel_9820\2835673221.py:2: DtypeWarning: Columns (13,24,102,104,106) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv("../data/output_data/soybeans_95_20.csv")
C:\Users\jhtchns2\AppData\Local\Temp\ipykernel_9820\2835673221.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  d['variety_id'] = d['variety_id'].str.replace(".","")


In [3]:
weather = pd.read_csv("../data/Illinois_Variety_Trials/rainfall.csv").iloc[:,:8]

weather.columns = ['location','ppt_april','ppt_may','ppt_june','ppt_july','ppt_aug','ppt_sept','year']

weather['location'] = weather.location.str.strip()\
                                      .str.lower()\
                                      .str.replace(" ","_")\
                                      .str.replace(".","")

weather = weather.set_index(["location",'year']).astype(float).reset_index()

for x in weather.columns[2:]:
    weather[x+"_sqr"] = weather[x]**2

d = d.merge(weather,on=['location','year'],how='left')

C:\Users\jhtchns2\AppData\Local\Temp\ipykernel_9820\2370693026.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  .str.replace(".","")


In [4]:
planting = pd.read_csv("../data/Illinois_Variety_Trials/harvest_dates.csv")

planting['location'] = planting['location'].str.strip()\
                                           .str.replace(" ","_")

planting['plant_date'] = pd.to_datetime(planting['plant_date'])
planting['harvest_date'] = pd.to_datetime(planting['harvest_date'])

planting['planting_week'] = \
[x.strftime("%V") if x!=-1 else np.nan for x in planting['plant_date'].fillna(-1)]

planting.loc[planting['planting_week']>="23","planting_week"] = ">= 23"

planting['harvest_week'] = \
[x.strftime("%V") if x!=-1 else np.nan for x in planting['harvest_date'].fillna(-1)]

d = d.merge(planting,on=['location','year'],how='left')

In [5]:
locs = pd.read_excel("../data/Illinois_Variety_Trials/plot_locations.xlsx")
w = pd.read_csv("../data/PRISM/out.csv")
w = w.drop_duplicates()

w['county'] = w['NAME'].str.lower()\
                       .str.replace(".","")

locs['county'] = locs['County'].str.lower()\
                               .str.replace(".","")

w['FIPS'] = w['STATEFP'].astype(str) + \
            w['COUNTYFP'].astype(str).str.zfill(3)

locs["FIPS"] = locs["FIPS"].astype(str)

w = w.merge(locs,on="FIPS")

w = w.set_index(['Location','year','month'])[['tmin','tmax','tmean']]
w = w.unstack()
w.columns = [x[0]+"_"+str(x[1]) for x in w.columns.values]
w = w.reset_index()
w['location'] = w['Location'].str.lower().str.replace(" ","_").str.replace(".","")

temp_cols = w.columns[2:-1]

d = d.merge(w,on=["location",'year'],how="left")

C:\Users\jhtchns2\AppData\Local\Temp\ipykernel_9820\73481558.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  .str.replace(".","")
C:\Users\jhtchns2\AppData\Local\Temp\ipykernel_9820\73481558.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  .str.replace(".","")
C:\Users\jhtchns2\AppData\Local\Temp\ipykernel_9820\73481558.py:22: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  w['location'] = w['Location'].str.lower().str.replace(" ","_").str.replace(".","")


In [6]:
d['maturity'] = d.maturity.astype(float)

# Maturity week
d['maturity_week'] = [x.strftime("%V") 
                          if x!=-1 
                          else np.nan 
                              for x in pd.to_datetime(d['maturity_date']).fillna(-1)]

# Seed treatment dummy
d['seed_treated'] = (d['seed_treat_entry'].fillna("U") !="U").astype(float)
d.loc[pd.isnull(d.seed_treat_entry),'seed_treated'] = np.nan

# Soybean cyst nematode resistance (S = susceptible)
d['scn_resist'] = (d.sn!="S").astype(float)
d.loc[pd.isnull(d.sn),'scn_resist'] = np.nan
d.loc[d.sn==".",'scn_resist'] = np.nan

# Public variety names
d['jack'] = d['variety'].str.lower() == "jack"

d['dwight'] = d['variety'].str.lower().str.contains("dwight")

d['williams'] = d['variety'].str.lower().str.contains("williams")

# Company panel variables
companies = d.groupby(["year","company_name"])['variety'].nunique().unstack()

company_counts_00_20 = companies.loc[2000:].count().sort_values()
companies_00_20 = company_counts_00_20[company_counts_00_20>18].index

company_counts_95_20 = companies.count().sort_values()
companies_95_20 = company_counts_95_20[company_counts_95_20>23].index

d['company_panel_95'] = d.company_name.isin(companies_95_20)
d['company_panel_00'] = d.company_name.isin(companies_00_20)

# Company participation in FIRST
d['il_and_first'] = (d['first_il'] == "IL_and_FIRST").astype(int)
company_status = d.groupby("company_name")['il_and_first'].max().reset_index()
d = d.merge(company_status,how='left',on="company_name",suffixes=["","_max"])

# First year variety appeared in trials
first_year = d.groupby("variety_id")['year'].min().reset_index()
first_year.columns = ['variety_id',"first_year"]
d = d.merge(first_year,how="left",on="variety_id")

In [8]:
d3 = d[(d.location!="region")&\
      ((d.maturity_range.astype(str).str.contains("3"))|\
       (d['maturity_range']==3)|\
       ((d['maturity']>2.9)&(d['maturity']<4)))]

d2 = d[(d.location!="region")&\
      ((d.maturity_range.astype(str).str.contains("2"))|\
       (d['maturity_range']==2)|\
       (d['mgroup'].astype(str).str.contains("2"))|
       ((d['maturity']>1.9)&(d['maturity']<3)))]

In [10]:
d.to_csv("../data/output_data/Full_Trial_Data.csv",index=False)
d2.to_csv("../data/output_data/MG2_Data.csv",index=False)
d3.to_csv("../data/output_data/MG3_Data.csv",index=False)

In [11]:
id_vars = ['year','first_year','region','location',"company_name",'variety_id',
           'company_panel_00','company_panel_95','first_il',"il_and_first_max","first_year"]
x_vars = ['type',"height",'lodging','maturity_week',\
          "seed_treated",'lodging_comb',"scn_resist",
          "GE","gddp10","ppt","soybean_rm12_JAN","corn_rm12_JAN",
          "planting_week","harvest_week",\
          "ppt_may","ppt_june","ppt_july","ppt_aug","ppt_sept",
          "ppt_may_sqr","ppt_june_sqr","ppt_july_sqr","ppt_aug_sqr","ppt_sept_sqr"]+ list(temp_cols)
cols = ["yield","yield_kg"] + id_vars + x_vars

private_vars = ['year','region','location','yield','yield_kg',"height",\
                    "lodging_comb","maturity_week",\
                    "seed_treated","scn_resist","planting_week","harvest_week"]

In [13]:
# Non-public varieties in the maturity group III
private3 = d3[~d3['public_variety']][cols]

# Williams yield only
W = d3[d3['williams']][private_vars]
# W = W.rename(columns={"yield":"williams_yield"})

# Merge to the private yields
private3 = private3.merge(W,on=['year','region','location'],how='left',suffixes=["","_w"])

# Subtract the difference
private3['diff_w'] = private3['yield'] - private3['yield_w']
private3['diff_kg_w'] = private3['yield_kg'] - private3['yield_kg_w']

# Non-public varieties in the maturity group II
private2 = d2[~d2['public_variety']][cols]



# Jack and Dwight yields
J = d2[d2['jack']][private_vars]

D = d2[d2['dwight']][private_vars]

# Merge and difference
private2 = private2.merge(J,on=['year','region','location'],how='left',suffixes=["","_j"])

private2['diff_j'] = private2['yield'] - private2['yield_j']
private2['diff_kg_j'] = private2['yield_kg'] - private2['yield_kg_j']


private2 = private2.merge(D,on=['year','region','location'],how='left',suffixes=["","_d"])

private2['diff_d'] = private2['yield'] - private2['yield_d']
private2['diff_kg_d'] = private2['yield_kg'] - private2['yield_kg_d']

In [14]:
private2.to_csv("../data/output_data/MG2_Private.csv",index=False)
private3.to_csv("../data/output_data/MG3_Private.csv",index=False)